In [1]:
# Import the required libraries
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle

In [2]:
# Load the data into pandas dataframes. The data has to be manually saved to a folder called 'data'.
# Note: the data is quite large, so this may take a while (~40 seconds) if the data is read as a csv. To speed up further file reading, it is converted to pickle format.
if (os.path.exists(os.path.join('data', 'training_set_VU_DM.pickle'))
    & os.path.exists((os.path.join('data', 'test_set_VU_DM.pickle')))):
    train_df = pd.read_pickle(os.path.join('data', 'training_set_VU_DM.pickle'))
    test_df = pd.read_pickle(os.path.join('data', 'test_set_VU_DM.pickle'))
else:
    train_df = pd.read_csv(os.path.join('data', 'training_set_VU_DM.csv'))
    test_df = pd.read_csv(os.path.join('data', 'test_set_VU_DM.csv'))
    train_df.to_pickle(os.path.join('data', 'training_set_VU_DM.pickle'))
    test_df.to_pickle(os.path.join('data', 'test_set_VU_DM.pickle'))

# train_df = pd.read_csv(os.path.join('data', 'dummy_training.csv'))
# test_df = pd.read_csv(os.path.join('data', 'dummy_testing.csv'))

In [3]:
print(f"Period of data collection: {pd.to_datetime(train_df['date_time']).min().strftime('%Y/%m/%d')} - {pd.to_datetime(train_df['date_time']).max().strftime('%Y/%m/%d')}")
print(f"Train data contains {train_df.shape[0]:,} rows and {train_df.shape[1]} columns")
print(f"Test data contains {test_df.shape[0]:,} rows and {test_df.shape[1]} columns")
print()
print(f"Train data:")
print(f"Number of unique search IDs: {len(train_df['srch_id'].unique()):,}")
print(f"Number of unique property IDs: {len(train_df['prop_id'].unique()):,}")
print(f"Number of clicks per search: avg. {train_df['click_bool'].sum() / len(train_df['srch_id'].unique()):.2f}, std. {train_df['click_bool'].std():.2f}")
print(f"Number of bookings per search: avg. {train_df['booking_bool'].sum() / len(train_df['srch_id'].unique()):.2f}, std. {train_df['booking_bool'].std():.2f}")
print()
print(f"Test data:")
print(f"Number of unique search IDs: {len(test_df['srch_id'].unique()):,}")
print(f"Number of unique property IDs: {len(test_df['prop_id'].unique()):,}")

Period of data collection: 2012/11/01 - 2013/06/30
Train data contains 4,958,347 rows and 54 columns
Test data contains 4,959,183 rows and 50 columns

Train data:
Number of unique search IDs: 199,795
Number of unique property IDs: 129,113
Number of clicks per search: avg. 1.11, std. 0.21
Number of bookings per search: avg. 0.69, std. 0.16

Test data:
Number of unique search IDs: 199,549
Number of unique property IDs: 129,438


In [4]:
def create_features(df):
    """
    Create the following new features: has_starrating, has_review_score, traveling_abroad, srch_prop_country_match, month, and day_of_week
    """
    # has_starrating: boolean whether prop_starrating is 0 or null
    df["has_starrating"] = df["prop_starrating"].isnull()
    df["has_starrating"] = df["has_starrating"].astype(int)
    df.loc[df["prop_starrating"] == 0, "has_starrating"] = 1

    # has_review_score: boolean whether prop_review_score is 0 or null
    df["has_review_score"] = df["prop_review_score"].isnull()
    df["has_review_score"] = df["has_review_score"].astype(int)
    df.loc[df["prop_review_score"] == 0, "has_review_score"] = 1

    # traveling_abroad: boolean whether visitor_location_country_id != prop_country_id
    df["traveling_abroad"] = df["visitor_location_country_id"] != df["prop_country_id"]
    df["traveling_abroad"] = df["traveling_abroad"].astype(int)

    # srch_prop_country_match: boolean whether srch_destination_id == prop_country_id
    df["srch_prop_country_match"] = df["srch_destination_id"] == df["prop_country_id"]
    df["srch_prop_country_match"] = df["srch_prop_country_match"].astype(int)

    # month: month of the search, one-hot encoded
    df["date_time"] = pd.to_datetime(df["date_time"])
    df["month"] = df["date_time"].dt.month
    df["month"] = df["month"].map({1: "jan", 2: "feb", 3: "mar", 4: "apr", 5: "may", 6: "jun", 7: "jul", 8: "aug", 9: "sep", 10: "oct", 11: "nov", 12: "dec"})
    df = pd.get_dummies(df, columns=["month"], dtype=int)
    for col in ["month_jan", "month_feb", "month_mar", "month_apr", "month_may", "month_jun", "month_jul", "month_aug", "month_sep", "month_oct", "month_nov", "month_dec"]:
        if col not in df.columns:
            df[col] = 0

    # day_of_week: day of the week of the search
    df["day_of_week"] = df["date_time"].dt.dayofweek
    df["day_of_week"] = df["day_of_week"].map({0: "mon", 1: "tue", 2: "wed", 3: "thu", 4: "fri", 5: "sat", 6: "sun"})
    df = pd.get_dummies(df, columns=["day_of_week"], dtype=int)
    for col in ["day_of_week_mon", "day_of_week_tue", "day_of_week_wed", "day_of_week_thu", "day_of_week_fri", "day_of_week_sat", "day_of_week_sun"]:
        if col not in df.columns:
            df[col] = 0

    return df

In [5]:
def drop_nan_columns(df):
    """
    Train data shows that "visitor_hist_starrating", "visitor_hist_adr_usd", "srch_query_affinity_score", and "compx_rate_percent_diff" have >90% NaN values. These values cannot be imputed accurately, so we drop these columns.
    """
    cols = ["visitor_hist_starrating", "visitor_hist_adr_usd", "srch_query_affinity_score"] + [f"comp{i}_rate_percent_diff" for i in range(1, 9)]
    df.drop(columns=cols, inplace=True)
    return df

In [6]:
def impute_missing_values(df):
    """
    Impute missing values for the following columns: "prop_starrating", "prop_review_score", "compx_rate", and "compx_inv".

    For "prop_starrating" and "prop_review_score", we replace 0 values with NaN and then impute the NaN values with the mean per srch_id. Remaining NaN values are then filled with 0.
    For "compx_rate" and "compx_inv", we assume that missing data means that Expedia has the same price and equal availability as its competitors. We therefore impute the NaN values with 0.
    """
    # Replace 0 values with NaN
    df["prop_starrating"] = df["prop_starrating"].replace(0, np.nan)
    df["prop_review_score"] = df["prop_review_score"].replace(0, np.nan)
    # Impute NaN values with mean per srch_id
    df["prop_starrating"] = df.groupby("srch_id")["prop_starrating"].transform(lambda x: x.fillna(x.mean()))
    df["prop_review_score"] = df.groupby("srch_id")["prop_review_score"].transform(lambda x: x.fillna(x.mean()))
    # Fill remaining NaN values with 0
    df["prop_starrating"] = df["prop_starrating"].fillna(0)
    df["prop_review_score"] = df["prop_review_score"].fillna(0)

    # Impute NaN values with 0
    for i in range(1, 9):
        df[f"comp{i}_rate"] = df[f"comp{i}_rate"].fillna(0)
        df[f"comp{i}_inv"] = df[f"comp{i}_inv"].fillna(0)

    return df

In [7]:
def compute_aggregated_values(df):
    """
    Compute the mean, median and standard deviation for the following columns:
    "visitor_hist_starrating", "visitor_hist_adr_usd", "prop_starrating", "prop_review_score", "prop_location_score1", "prop_log_historical_price", "price_usd"
    """
    numerical_cols = ["prop_starrating", "prop_review_score", "prop_location_score1", "prop_log_historical_price", "price_usd"]
    # srch_length_of_stay, srch_booking_window, srch_adults_count, srch_children_count, and srch_room_count are also numerical variables, but it has no use aggregating these values over prop_id.

    agg_df = df.groupby("prop_id").agg({col: ["mean", "std", "median"] for col in numerical_cols})
    agg_df.columns = ["_".join(col) for col in agg_df.columns]
    agg_df.fillna(0, inplace=True)  # Fill standard deviation NaN values with 0
    for col in agg_df.columns:
        df[col] = df["prop_id"].map(agg_df[col])
    return df

In [8]:
def compute_relative_values(df):
    """
    Subtract the mean per srch_id from the following columns:
    "prop_starrating", "prop_review_score", "prop_location_score1", "prop_log_historical_price", "price_usd"

    This is done so that the model can learn the relative values of these columns per srch_id.
    """
    cols = ["prop_starrating", "prop_review_score", "prop_location_score1", "prop_log_historical_price", "price_usd"]
    grouper = df.groupby('srch_id')
    for col in cols:
        df[col] = df[col] - grouper[col].transform('mean')
    return df

In [9]:
def drop_columns(df):
    """
    Train data shows that for "orig_destination_distance" over 75% of the data with a calculated value lower than 0.95 of the largest distance was lower than 130, meaning that the distance per srch_id is roughly the same. We assume therefore that this is not a deciding factor for a customer in their booking process and drop this column.

    Features were created from "date_time" and the column will not be used anymore, so we drop this column as well.

    Columns containing IDs ("site_id", "visitor_location_country_id", "prop_country_id", "prop_id", and "srch_destination_id") are not used in the model, so we drop these columns as well. "srch_id" and "prop_id" will remain in the columns for now for later use.

    If the supplied dataframe is the training dataframe, drop the unused target columns as well.

    # TODO:
    I don't really know what to do with "prop_location_score2" yet, so I'll drop it for now.
    """
    df.drop(columns=["orig_destination_distance"], inplace=True)
    df.drop(columns=["date_time"], inplace=True)
    df.drop(columns=["site_id", "visitor_location_country_id", "prop_country_id", "srch_destination_id"], inplace=True)
    df.drop(columns=["prop_location_score2"], inplace=True)
    for col in ["position", "gross_bookings_usd"]:
        if col in df.columns:
            df.drop(columns=[col], inplace=True)
    return df

In [10]:
def process_data(train_df: pd.DataFrame, test_df: pd.DataFrame, target_value: str = "booking_bool"):
    """
    Process the dataframes for training and testing the model.
    :param train_df: The training dataframe
    :param test_df: The test dataframe
    :param target_value: The target value to use for training the model. Either "booking_bool" or "both".
    :return: The processed dataframes
    """
    # Create features
    train_df = create_features(train_df)
    test_df = create_features(test_df)

    # Drop columns with NaN values
    train_df = drop_nan_columns(train_df)
    test_df = drop_nan_columns(test_df)

    # Impute missing values
    train_df = impute_missing_values(train_df)
    test_df = impute_missing_values(test_df)

    # Compute aggregated values
    train_df = compute_aggregated_values(train_df)
    test_df = compute_aggregated_values(test_df)

    # Compute relative values
    train_df = compute_relative_values(train_df)
    test_df = compute_relative_values(test_df)

    # Drop columns
    train_df = drop_columns(train_df)
    test_df = drop_columns(test_df)

    # Split train data into features and target
    if target_value == "booking_bool":
        train_df["target"] = train_df["booking_bool"]
    elif target_value == "both":
        train_df["target"] = 5*train_df["booking_bool"] + train_df["click_bool"]
    else:
        raise ValueError("target_value must be either 'booking_bool' or 'both'")

    train_df.drop(columns=["booking_bool", "click_bool"], inplace=True)

    return train_df, test_df

In [11]:
def generate_sequences(df: pd.DataFrame, kind: str = "train") -> dict:
    """
    Generate sequences from the dataframe.
    :param df: The dataframe to take the sequences from.
    :param kind: The kind of dataframe. Either "train" or "test".
    :return: Dictionary containing the sequences and targets for the training data.
    """
    sequences = []
    if kind == "train":
        idx_dict = df.to_dict("index")
        df = df.drop(columns=["prop_id", "srch_id", "target"])
        for i, idx in enumerate(tqdm(idx_dict, desc="Train DataLoader")):
            sequences.append({"srch_id": idx_dict[idx]["srch_id"], "prop_id": idx_dict[idx]["prop_id"], "sequence": df.iloc[i].values, "target": idx_dict[idx]["target"]})
    elif kind == "test":
        idx_dict = df.to_dict("index")
        df = df.drop(columns=["prop_id", "srch_id"])
        for i, idx in enumerate(tqdm(idx_dict, desc="Test DataLoader")):
            sequences.append({"srch_id": idx_dict[idx]["srch_id"], "prop_id": idx_dict[idx]["prop_id"], "sequence": df.iloc[i].values})
    else:
        raise ValueError("kind must be either 'train' or 'test'")
    return sequences

In [12]:
def generate_lgb_sequences(train_df, test_df, force_data_processing=False):
    """
    Generate the train and test dictionaries
    """
    if os.path.exists(os.path.join('data', f'train_sequences_both.pickle')) \
            & os.path.exists(os.path.join('data', f'test_sequences_both.pickle')) \
            & (not force_data_processing):
        print(f"Loading train and test sequences from pickle files...")
        with open(os.path.join('data', f'train_sequences_both.pickle'), 'rb') as f:
            train_sequences = pickle.load(f)
        with open(os.path.join('data', f'test_sequences_both.pickle'), 'rb') as f:
            test_sequences = pickle.load(f)
    else:
        processed_train_df, processed_test_df = process_data(train_df, test_df, target_value="both")
        train_sequences = generate_sequences(processed_train_df, kind="train")
        with open(os.path.join('data', f'train_sequences_both.pickle'), 'wb') as f:
            pickle.dump(train_sequences, f)
        test_sequences = generate_sequences(processed_test_df, kind="test")
        with open(os.path.join('data', f'test_sequences_both.pickle'), 'wb') as f:
            pickle.dump(test_sequences, f)

    return train_sequences, test_sequences

def generate_lgb_data(sequences, kind="train"):
    train_x = np.array([seq["sequence"] for seq in sequences])
    
    query_to_properties = {}
    for seq in sequences:
        
        srch_id = seq["srch_id"]
        prop_id = seq["prop_id"]
        # Add the search ID to the dictionary if it isn't in already
        if srch_id not in query_to_properties.keys():
            query_to_properties[srch_id] = []
        
        query_to_properties[srch_id].append(prop_id)

    # When generating train data, return also targets
    if kind=="train":
        train_y = np.array([seq["target"] for seq in sequences])

        return train_x, train_y, query_to_properties
    
    # When generating test data, return only features and query information
    elif kind=="test":

        return train_x, query_to_properties

    else:
        raise ValueError("Invalid dataset creation")

**Train the LambdaMART model**

In [13]:
import lightgbm as lgb
import time

# Assume X_train and y_train are the training features and target values, respectively
print("Generating sequences...")
t = time.time()
lgb_train_sequences, lgb_test_sequences = generate_lgb_sequences(train_df, test_df)
print(f"Done generating sequences. This took {time.time() - t:.2f}s")

t = time.time()
print("Generating train features...")
train_x, train_y, train_queries = generate_lgb_data(lgb_train_sequences, kind="train")
print(f"Done generating train features. This took {time.time() - t:.2f}s")

t = time.time()
print("Transforming train features into appropriate dataset...")
# Create a LightGBM dataset from the training data
train_data = lgb.Dataset(train_x, label=train_y, group=[len(elem) for elem in train_queries.values()])
print(f"Done generating dataset. This took {time.time() - t:.2f}s")

# Set the parameters for the LambdaMART model
params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'ndcg_eval_at': [5],  # Evaluation at NDCG@5
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'device': 'gpu' # Change to GPU if available
}

# Train the LambdaMART model
num_rounds = 100

t = time.time()
print("Training model...")
model = lgb.train(params, train_data, num_rounds)
print(f"Done training model. This took {time.time() - t:.2f}s")

# Save the trained model
model.save_model('lambda_mart_model.txt')

Generating sequences...
Loading train and test sequences from pickle files...
Done generating sequences. This took 45.46s
Generating train features...
Done generating train features. This took 26.58s
Transforming train features into appropriate dataset...
Done generating dataset. This took 0.04s
Training model...
Done training model. This took 36.75s


**Test the LambdaMART model and save the output**

In [15]:
def test_and_write(fn, predictions, queries):

    srch_to_ranks = {}
    
    i=0
    for query_id, property_ids in tqdm(queries.items()):
        
        # Retrieve all probabilities for a certain query ID (assuming the data is still ordered)
        ranks = predictions[i:i+len(property_ids)]

        # Keep track of the probabilities
        srch_to_ranks[query_id] = ranks
        i+=len(property_ids)
    
    # Write predictions to file
    with open(fn, "w") as f:
        f.write("srch_id,prop_id\n")

        # Iterate over all different queries
        for srch_id, srch_probs in srch_to_ranks.items():
            
            # Get the order of the hotels based on the predicted probabilities
            sorted_indices = np.array(srch_probs).argsort()[::-1]
            property_ids_sorted = np.array(queries[srch_id])[np.array(sorted_indices)]

            for s_id, p_id in zip([srch_id for _ in range(len(property_ids_sorted))],property_ids_sorted):
                f.write(f"{s_id},{p_id}\n")

In [16]:
# Get the test data
test_x, test_queries = generate_lgb_data(lgb_test_sequences, kind="test")

predictions = model.predict(test_x, raw_score=False)

# Print the predictions
print(predictions)

# Write predictions to file
test_and_write("test_output/LambdaMART_vanilla.csv", predictions, test_queries)

[-0.22006296 -0.2416695  -0.30013754 ... -0.72995487 -0.63153378
 -0.24392625]


100%|██████████| 199549/199549 [00:00<00:00, 1124304.23it/s]
